<a href="https://colab.research.google.com/github/Neejay10122004/Predictions/blob/main/Scream_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os

import torchaudio
import torch
import matplotlib.pyplot as plt
from pathlib import Path

# Load the data

In [ ]:
def load_audio_files(path: str, label: str):
    dataset = []

    walker = sorted(str(p) for p in Path(path).glob('*.wav'))

    for i, file_path in enumerate(walker):
        path, filename = os.path.split(file_path)
        name, _ = os.path.splitext(filename)

        waveform, sample_rate = torchaudio.load(file_path)

        entry = {'waveform': waveform, 'sample_rate': sample_rate, 'label': label, 'id': i}

        dataset.append(entry)

    return dataset

In [ ]:
screaming_dataset = load_audio_files('/kaggle/input/human-screaming-detection-dataset/Screaming', 'yes')

screaming_dataset[:5]

[]

In [ ]:
not_screaming_dataset = load_audio_files('/kaggle/input/human-screaming-detection-dataset/NotScreaming', 'not')

not_screaming_dataset[:5]

[]

In [ ]:
display(len(screaming_dataset), len(not_screaming_dataset))

0

0

# Check the sample of the data

In [ ]:
scream_id = 5
no_scream_id = 0

screaming_waveform = screaming_dataset[scream_id]['waveform']
screaming_sample_rate = screaming_dataset[scream_id]['sample_rate']

print(f'Screaming Waveform: {screaming_waveform}')
print(f'Screaming Sample Rate: {screaming_sample_rate}')
print(f'Screaming Label: {screaming_dataset[scream_id]["label"]}')
print(f'Screaming ID: {screaming_dataset[scream_id]["id"]} \n')

not_screaming_waveform = not_screaming_dataset[no_scream_id]['waveform']
not_screaming_sample_rate = not_screaming_dataset[no_scream_id]['sample_rate']

print(f'Screaming Waveform: {not_screaming_waveform}')
print(f'Screaming Sample Rate: {not_screaming_sample_rate}')
print(f'Screaming Label: {not_screaming_dataset[no_scream_id]["label"]}')
print(f'Screaming ID: {not_screaming_dataset[no_scream_id]["id"]} \n')

Error: scream_id 5 is out of range. Valid indices are 0 to -1.
Error: no_scream_id 0 is out of range. Valid indices are 0 to -1.


In [ ]:
def show_waveform(waveform, sample_rate, label):
    print("Waveform: {}\nSample rate: {}\nLabels: {} \n".format(waveform, sample_rate, label))

    new_sample_rate = sample_rate / 10

    channel = 0

    waveform_transformed = torchaudio.transforms.Resample(sample_rate, new_sample_rate)(waveform[channel, :].view(1, -1))

    print("Shape of transformed waveform: {}\nSample rate: {}".format(waveform_transformed.size(), new_sample_rate))

    plt.figure()
    plt.plot(waveform_transformed[0, :].numpy())

In [ ]:
show_waveform(screaming_waveform, screaming_sample_rate, 'yes')

NameError: name 'screaming_waveform' is not defined

In [ ]:
import IPython.display as ipd

ipd.Audio(screaming_waveform.numpy(), rate=screaming_sample_rate)

In [ ]:
show_waveform(not_screaming_waveform, not_screaming_sample_rate, 'not')

In [ ]:
ipd.Audio(not_screaming_waveform.numpy(), rate=not_screaming_sample_rate)

# View with spectrogram

In [ ]:
def show_spectrogram(waveform_classA, waveform_classB):
    yes_spectrogram = torchaudio.transforms.Spectrogram()(waveform_classA)
    print("\nShape of yes spectrogram: {}".format(yes_spectrogram.size()))

    no_spectrogram = torchaudio.transforms.Spectrogram()(waveform_classB)
    print("Shape of no spectrogram: {}".format(no_spectrogram.size()))

    plt.figure()

    plt.subplot(1, 2, 1)
    plt.title("Features of {}".format('yes'))

    plt.imshow(yes_spectrogram.log2()[0, :, :].numpy(), cmap='viridis', aspect='auto')

    plt.subplot(1, 2, 2)
    plt.title("Features of {}".format('no'))

    plt.imshow(no_spectrogram.log2()[0, :, :].numpy(), cmap='viridis', aspect='auto')

In [ ]:
show_spectrogram(screaming_waveform, not_screaming_waveform)

In [ ]:
def show_mel_spectrogram(waveform, sample_rate):
    mel_spectrogram = torchaudio.transforms.MelSpectrogram(
            sample_rate=sample_rate,
            n_mels=64,
            n_fft=1024
    )(waveform)

    print("Shape of spectrogram: {}".format(mel_spectrogram.size()))

    plt.figure()

    plt.imshow(mel_spectrogram.log2()[0, :, :].numpy(), cmap='viridis', aspect='auto')

In [ ]:
show_mel_spectrogram(screaming_waveform, screaming_sample_rate)

In [ ]:
show_mel_spectrogram(not_screaming_waveform, not_screaming_sample_rate)

In [ ]:
def show_mfcc(waveform, sample_rate):
    mfcc_spectrogram = torchaudio.transforms.MFCC(
            sample_rate=sample_rate,
            n_mfcc=128
    )(waveform)

    print("Shape of spectrogram: {}".format(mfcc_spectrogram.size()))

    plt.figure()

    plt.imshow(mfcc_spectrogram.log2()[0, :, :].numpy(), cmap='viridis', aspect='auto')

    plt.figure()
    plt.plot(mfcc_spectrogram.log2()[0, :, :].numpy())
    plt.draw()

In [ ]:
show_mfcc(screaming_waveform,  screaming_sample_rate)

In [ ]:
show_mfcc(not_screaming_waveform,  not_screaming_sample_rate)

# Preprogress for the audio

In [ ]:
def pad_waveform(waveform, target_length):
    _, num_channels, current_length = waveform.shape

    if current_length < target_length:
        padding = target_length - current_length
        waveform = torch.nn.functional.pad(waveform, (0, padding))

    return waveform


def create_images(train_loader, label_dir, amplitude_threshold=0.01):
    directory = f'Data/Images/{label_dir}/'
    if os.path.isdir(directory):
        print("Data exists for", label_dir)
    else:
        os.makedirs(directory, mode=0o777, exist_ok=True)

        for i, data in enumerate(train_loader):
            waveform = data['waveform']

            waveform = pad_waveform(waveform, 441000)

            if torch.max(torch.abs(waveform)) > amplitude_threshold:
                spectrogram_tensor = torchaudio.transforms.MelSpectrogram(
                    sample_rate=int(data['sample_rate']),
                    n_mels=64,
                    n_fft=1024,
                )(waveform)

                plt.imsave(f'Data/Images/{label_dir}/audio_img{i}.png', (spectrogram_tensor[0] + 1e-10).log2()[0, :, :].numpy(), cmap='viridis')
            else:
                print(f'Skipping blank waveform {i} in {label_dir}')

In [ ]:
from torch.utils.data import DataLoader

train_loader_scream = DataLoader(screaming_dataset, batch_size=1,
                                              shuffle=False, num_workers=0)
train_loader_not_scream = DataLoader(not_screaming_dataset, batch_size=1,
                                             shuffle=False, num_workers=0)

In [ ]:
create_images(train_loader_scream, 'scream')

In [ ]:
create_images(train_loader_not_scream, 'not')

# Optional Noise

In [ ]:
from torchvision.transforms import Lambda
import torch

def random_time_shift(audio, max_shift_ms=1000, target_length=441000):
    current_length = audio.shape[-1]

    if current_length >= target_length:
        shift = torch.randint(-max_shift_ms, max_shift_ms, (1,)).item()
        shift_samples = int(shift * 44100 / 1000)  # Assuming a 44.1 kHz sample rate
        if shift_samples >= 0:
            audio = torch.nn.functional.pad(audio, (shift_samples, 0))
            audio = audio[:, :target_length]
        else:
            audio = torch.nn.functional.pad(audio, (0, -shift_samples))
            audio = audio[:, -target_length:]
    else:
        padding = target_length - current_length
        shift_samples = torch.randint(1, padding, (1,)).item()

        left_padding = torch.randint(0, shift_samples, (1,)).item()

        right_padding = padding - left_padding

        audio = torch.nn.functional.pad(audio, (left_padding, right_padding))

    return audio

In [ ]:
def add_noise(audio, noise_level=0.005):
    noise = noise_level * torch.randn_like(audio)
    noisy_audio = audio + noise
    return noisy_audio

In [ ]:
from torchvision.transforms import Compose

def augment_audio(audio):
    transform_audio = Compose([
        Lambda(lambda x: random_time_shift(x))
    ])

    augmented_audio = transform_audio(audio)

    return augmented_audio

In [ ]:
shift_audio = augment_audio(screaming_waveform)
shift_audio.shape

In [ ]:
show_waveform(shift_audio, screaming_sample_rate, 'Test')

In [ ]:
def create_shift_images(train_loader, label_dir, amplitude_threshold=0.01, shift_time=0):
    for i, data in enumerate(train_loader):
        waveform = data['waveform']

        waveform = random_time_shift(waveform)

        if torch.max(torch.abs(waveform)) > amplitude_threshold:
            spectrogram_tensor = torchaudio.transforms.MelSpectrogram(
                sample_rate=int(data['sample_rate']),
                n_mels=64,
                n_fft=1024,
            )(waveform)

            if spectrogram_tensor[0].log2().isnan().any() or spectrogram_tensor[0].log2().isinf().any():
                continue

            plt.imsave(f'Data/Images/{label_dir}/audio_img{i}_shift{shift_time}.png', spectrogram_tensor[0].log2()[0, :, :].numpy(), cmap='viridis')
        else:
            print(f'Skipping blank waveform {i} in {label_dir}')

    return 'Done!'

# To balance the data for positive we add random shift or noise or even both to increase the data size

In [ ]:
[create_shift_images(train_loader_scream, 'scream', shift_time=i) for i in range(5)]